# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
#items_df = pd.read_csv(data_dir + '/ml-20m/movies.csv', delimiter='\t', index_col=0)
items_df = pd.read_csv(data_dir + '/ml-25m/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [5]:
movie_id_example = 4
title = items_df.loc[movie_id_example]['title']
print(title)

Waiting to Exhale (1995)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining artist"

Now let's test a few simple values to check our error catching.

In [7]:
# A known good id
print(get_movie_by_id(movie_id="987"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Bliss (1997)
Error obtaining artist
Error obtaining artist


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is an artist. 

So, let's sample some data from our dataset to test our SIMS campaign. Grab 5 random artists from our dataframe.

In [8]:
samples = items_df.sample(5)
samples

,title
movieId,
123808,The Conquering Power (1921)
142532,Call Girl of Cthulhu (2014)
116949,Thrust in Me (1985)
161004,Russian Holiday (1992)
5207,"Mountain Men, The (1980)"


@TODO

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Earth, Wind & Fire).

In [9]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(912),
)

In [10]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Citizen Kane (1941)
Rear Window (1954)
North by Northwest (1959)
Maltese Falcon, The (1941)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Graduate, The (1967)
African Queen, The (1951)
Godfather: Part II, The (1974)
One Flew Over the Cuckoo's Nest (1975)
To Kill a Mockingbird (1962)
Chinatown (1974)
Bridge on the River Kwai, The (1957)
Vertigo (1958)
Wizard of Oz, The (1939)
Gone with the Wind (1939)
Lawrence of Arabia (1962)
Psycho (1960)
Godfather, The (1972)
Amadeus (1984)
It's a Wonderful Life (1946)
Singin' in the Rain (1952)
Annie Hall (1977)
Apocalypse Now (1979)
Sting, The (1973)
12 Angry Men (1957)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [11]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with the sample of artists we chose before.

In [12]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,The Conquering Power (1921),Call Girl of Cthulhu (2014),Thrust in Me (1985),Russian Holiday (1992),"Mountain Men, The (1980)"
0,"Shawshank Redemption, The (1994)",Tall Men (2016),Turbo Kid (2015),"Shawshank Redemption, The (1994)",Gods and Generals (2003)
1,Forrest Gump (1994),Ghosts of Darkness (2017),NaN,Forrest Gump (1994),Nevada Smith (1966)
2,Pulp Fiction (1994),The Babysitter (2017),NaN,Pulp Fiction (1994),Rio Lobo (1970)
3,"Silence of the Lambs, The (1991)","Dead, The (2010)",NaN,"Silence of the Lambs, The (1991)","Man Called Horse, A (1970)"
4,"Matrix, The (1999)",Deadgirl (2008),NaN,"Matrix, The (1999)",Rough Night in Jericho (1967)
5,Star Wars: Episode IV - A New Hope (1977),"Congress, The (2013)",NaN,Star Wars: Episode IV - A New Hope (1977),Bite the Bullet (1975)
6,Schindler's List (1993),Pride and Prejudice and Zombies (2016),NaN,Schindler's List (1993),Breakheart Pass (1975)
7,Jurassic Park (1993),Superman/Shazam!: The Return of Black Adam (2010),NaN,Jurassic Park (1993),Mackenna's Gold (1969)
8,Braveheart (1995),Ultimate Avengers 2 (2006),NaN,Braveheart (1995),Sands of Iwo Jima (1949)
9,Fight Club (1999),Batman: Return of the Caped Crusaders (2016),NaN,Fight Club (1999),She Wore a Yellow Ribbon (1949)


You may notice that a lot of the items look the same, hopefully not all of them do. This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### HRNN

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [13]:
#users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
#users_df = pd.read_csv(data_dir + '/ml-25m/users.csv', sep=',', usecols=[0,1], encoding='latin-1', index_col=0)

# Render some sample data
#users_df.head(5)

In [14]:
#users = users_df.sample(3).index.tolist()
users = [
    '2010',
    '13245',
    '5626',
    '939',
    '78877'
]
users

['2010', '13245', '5626', '939', '78877']

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [15]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [16]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()
print(users)

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

['2010', '13245', '5626', '939', '78877']


,2010,13245,5626,939,78877
0,Young Guns II (1990),Fargo (1996),Interview with the Vampire: The Vampire Chroni...,Tomb Raider (2018),"Truth About Cats & Dogs, The (1996)"
1,Young Guns (1988),Jurassic Park (1993),"Net, The (1995)",Pacific Rim: Uprising (2018),Mighty Aphrodite (1995)
2,Tombstone (1993),Forrest Gump (1994),Waterworld (1995),Rampage (2018),Phenomenon (1996)
3,Maverick (1994),Speed (1994),Natural Born Killers (1994),Jurassic World: Fallen Kingdom (2018),Star Trek: First Contact (1996)
4,Wyatt Earp (1994),Schindler's List (1993),While You Were Sleeping (1995),Red Sparrow (2018),"Nutty Professor, The (1996)"
5,Unforgiven (1992),Dolores Claiborne (1995),Braveheart (1995),Solo: A Star Wars Story (2018),"Cable Guy, The (1996)"
6,Â¡Three Amigos! (1986),Heat (1995),GoldenEye (1995),Bright (2017),"Postman, The (Postino, Il) (1994)"
7,City Slickers II: The Legend of Curly's Gold (...,"Piano, The (1993)",Quiz Show (1994),Jumanji: Welcome to the Jungle (2017),"River Wild, The (1994)"
8,Last Man Standing (1996),Muriel's Wedding (1994),"Firm, The (1993)",Ant-Man and the Wasp (2018),Star Wars: Episode IV - A New Hope (1977)
9,Legends of the Fall (1994),Desperado (1995),Outbreak (1995),Aquaman (2018),Nixon (1995)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

Now lets apply item filters to see recommendations for each of these users within a genre


In [17]:
def get_new_action_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [18]:
user

'78877'

In [20]:
recommendations_df_genres = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_genres = get_new_action_recommendations_df_by_filter(recommendations_df_genres, user, filter_arn)
    
    
recommendations_df_genres

,Adventure,Horror,Fantasy,Documentary,Thriller,Adventure,Horror,Fantasy,Documentary,Thriller,Action,Mystery,Romance
0,Star Trek: First Contact (1996),Dracula: Dead and Loving It (1995),"Nutty Professor, The (1996)",Heidi Fleiss: Hollywood Madam (1995),Star Trek: First Contact (1996),Star Trek: First Contact (1996),Dracula: Dead and Loving It (1995),"Nutty Professor, The (1996)",Heidi Fleiss: Hollywood Madam (1995),Star Trek: First Contact (1996),Star Trek: First Contact (1996),Primal Fear (1996),"Truth About Cats & Dogs, The (1996)"
1,Star Wars: Episode IV - A New Hope (1977),"Craft, The (1996)",Dragonheart (1996),Crumb (1994),"Cable Guy, The (1996)",Star Wars: Episode IV - A New Hope (1977),"Craft, The (1996)",Dragonheart (1996),Crumb (1994),"Cable Guy, The (1996)","River Wild, The (1994)",Mission: Impossible (1996),Mighty Aphrodite (1995)
2,Rumble in the Bronx (Hont faan kui) (1995),Scream (1996),James and the Giant Peach (1996),Anne Frank Remembered (1995),"River Wild, The (1994)",Rumble in the Bronx (Hont faan kui) (1995),Scream (1996),James and the Giant Peach (1996),Anne Frank Remembered (1995),"River Wild, The (1994)",Star Wars: Episode IV - A New Hope (1977),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Phenomenon (1996)
3,Dragonheart (1996),"Frighteners, The (1996)",Willy Wonka & the Chocolate Factory (1971),"Celluloid Closet, The (1995)","Time to Kill, A (1996)",Dragonheart (1996),"Frighteners, The (1996)",Willy Wonka & the Chocolate Factory (1971),"Celluloid Closet, The (1995)","Time to Kill, A (1996)",Rumble in the Bronx (Hont faan kui) (1995),Scream (1996),"Nutty Professor, The (1996)"
4,Star Wars: Episode VI - Return of the Jedi (1983),"Silence of the Lambs, The (1991)",Toy Story (1995),Catwalk (1996),Primal Fear (1996),Star Wars: Episode VI - Return of the Jedi (1983),"Silence of the Lambs, The (1991)",Toy Story (1995),Catwalk (1996),Primal Fear (1996),Dragonheart (1996),Lone Star (1996),"Postman, The (Postino, Il) (1994)"
5,James and the Giant Peach (1996),Heavy Metal (1981),"Craft, The (1996)",Hoop Dreams (1994),"Juror, The (1996)",James and the Giant Peach (1996),Heavy Metal (1981),"Craft, The (1996)",Hoop Dreams (1994),"Juror, The (1996)",Star Wars: Episode VI - Return of the Jedi (1983),"City of Lost Children, The (CitÃ© des enfants ...",Sabrina (1995)
6,Executive Decision (1996),Vampire in Brooklyn (1995),Space Jam (1996),Looking for Richard (1996),Ransom (1996),Executive Decision (1996),Vampire in Brooklyn (1995),Space Jam (1996),Looking for Richard (1996),Ransom (1996),Eraser (1996),Strange Days (1995),Up Close and Personal (1996)
7,Independence Day (a.k.a. ID4) (1996),From Dusk Till Dawn (1996),Snow White and the Seven Dwarfs (1937),Paradise Lost: The Child Murders at Robin Hood...,Eraser (1996),Independence Day (a.k.a. ID4) (1996),From Dusk Till Dawn (1996),Snow White and the Seven Dwarfs (1937),Paradise Lost: The Child Murders at Robin Hood...,Eraser (1996),Sudden Death (1995),"Usual Suspects, The (1995)","Hunchback of Notre Dame, The (1996)"
8,"Rock, The (1996)",Mary Reilly (1996),Michael (1996),When We Were Kings (1996),City Hall (1996),"Rock, The (1996)",Mary Reilly (1996),Michael (1996),When We Were Kings (1996),City Hall (1996),Executive Decision (1996),Devil in a Blue Dress (1995),Tin Cup (1996)
9,Muppet Treasure Island (1996),Body Snatchers (1993),"City of Lost Children, The (CitÃ© des enfants ...","Endless Summer 2, The (1994)",Executive Decision (1996),Muppet Treasure Island (1996),Body Snatchers (1993),"City of Lost Children, The (CitÃ© des enfants ...","Endless Summer 2, The (1994)",Executive Decision (1996),Barb Wire (1996),"Client, The (1994)",Jerry Maguire (1996)


In [21]:
recommendations_df_genres = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_genres = get_new_action_recommendations_df_by_filter(recommendations_df_genres, user, filter_arn)
    
    
recommendations_df_genres

,watched,unwatched
0,Phenomenon (1996),"Truth About Cats & Dogs, The (1996)"
1,Star Trek: First Contact (1996),Mighty Aphrodite (1995)
2,"Cable Guy, The (1996)","Nutty Professor, The (1996)"
3,Nixon (1995),"Postman, The (Postino, Il) (1994)"
4,Happy Gilmore (1996),"River Wild, The (1994)"
5,"Time to Kill, A (1996)",Star Wars: Episode IV - A New Hope (1977)
6,Primal Fear (1996),Rumble in the Bronx (Hont faan kui) (1995)
7,"Juror, The (1996)",Dragonheart (1996)
8,Trainspotting (1996),Sabrina (1995)
9,Striptease (1996),Eraser (1996)


The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [22]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:059124553121:event-tracker/c87ed6d0
8f2108c5-92c9-492f-99ec-61d9576cdbdb


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [23]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with the artist
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [24]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,78877
0,"Truth About Cats & Dogs, The (1996)"
1,Mighty Aphrodite (1995)
2,Phenomenon (1996)
3,Star Trek: First Contact (1996)
4,"Nutty Professor, The (1996)"
5,"Cable Guy, The (1996)"
6,"Postman, The (Postino, Il) (1994)"
7,"River Wild, The (1994)"
8,Star Wars: Episode IV - A New Hope (1977)
9,Nixon (1995)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [25]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [26]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie)
    time.sleep(5)
user_recommendations_df

,78877,Four Days In November (1964),Alien Planet (2005),Girls on Top 2 (2004)
0,"Truth About Cats & Dogs, The (1996)","Truth About Cats & Dogs, The (1996)","Truth About Cats & Dogs, The (1996)",Star Wars: Episode VI - Return of the Jedi (1983)
1,Mighty Aphrodite (1995),Mighty Aphrodite (1995),Mighty Aphrodite (1995),"Godfather, The (1972)"
2,Phenomenon (1996),Phenomenon (1996),Phenomenon (1996),Star Wars: Episode IV - A New Hope (1977)
3,Star Trek: First Contact (1996),Star Trek: First Contact (1996),Star Trek: First Contact (1996),Star Trek: First Contact (1996)
4,"Nutty Professor, The (1996)","Nutty Professor, The (1996)","Nutty Professor, The (1996)",Forrest Gump (1994)
5,"Cable Guy, The (1996)","Cable Guy, The (1996)","Cable Guy, The (1996)",Toy Story (1995)
6,"Postman, The (Postino, Il) (1994)","Postman, The (Postino, Il) (1994)","Postman, The (Postino, Il) (1994)","Silence of the Lambs, The (1991)"
7,"River Wild, The (1994)","River Wild, The (1994)","River Wild, The (1994)","Shawshank Redemption, The (1994)"
8,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977),James and the Giant Peach (1996)
9,Nixon (1995),Nixon (1995),Nixon (1995),Trainspotting (1996)


In the cell above, the first column after the index is the user's default recommendations from HRNN, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset. If you wanted to better understand this, try simulating clicking random artists of random genres, and you should see a more pronounced impact.

### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. To demonstrate this, we will need a random user and a random collection of 25 items.

In [27]:
rerank_user = random.sample(users, 1)
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [28]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,5626
0,Bravissimo (1955)
1,"Darkest Hour, The (2011)"
2,Whatever It Takes (2000)
3,Purpose (2002)
4,Fitzcarraldo (1982)
5,The Mad Dog Killer (1977)
6,Ikingut (2000)
7,The Sorcerers (1967)
8,The Boss Baby (2017)
9,El rey de los huevones (2006)


Then make the personalized ranking API call.

In [29]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'd5228c0c-8d17-4e4e-a92d-35117e62ee23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 03 Sep 2020 21:31:54 GMT',
   'x-amzn-requestid': 'd5228c0c-8d17-4e4e-a92d-35117e62ee23',
   'content-length': '1450',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '6373', 'score': 0.5367461},
  {'itemId': '4816', 'score': 0.2541563},
  {'itemId': '168418', 'score': 0.0951344},
  {'itemId': '61240', 'score': 0.0647921},
  {'itemId': '64716', 'score': 0.0191426},
  {'itemId': '2970', 'score': 0.0121235},
  {'itemId': '91660', 'score': 0.0049147},
  {'itemId': '190183', 'score': 0.0048849},
  {'itemId': '3131', 'score': 0.0025817},
  {'itemId': '173377', 'score': 0.0012243},
  {'itemId': '206927', 'score': 0.0008446},
  {'itemId': '125155', 'score': 0.0007765},
  {'itemId': '152607', 'score': 0.0007333},
  {'itemId': '6606', 'score': 0.0006911},
  {'itemId': '138914', 'score

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [30]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,"(5626,)",Re-Ranked
0,Bravissimo (1955),Bruce Almighty (2003)
1,"Darkest Hour, The (2011)",Zoolander (2001)
2,Whatever It Takes (2000),The Boss Baby (2017)
3,Purpose (2002),Let the Right One In (LÃ¥t den rÃ¤tte komma in...
4,Fitzcarraldo (1982),Seven Pounds (2008)
5,The Mad Dog Killer (1977),Fitzcarraldo (1982)
6,Ikingut (2000),"Darkest Hour, The (2011)"
7,The Sorcerers (1967),The Darkest Minds (2018)
8,The Boss Baby (2017),Broadway Damage (1997)
9,El rey de los huevones (2006),Life Risking Romance (2016)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example, or if you are filtering on a category and want to show the most likely good items.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [31]:
# Get the user list
batch_users = random.sample(users,3)

# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [32]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "5626"}
{"userId": "2010"}
{"userId": "78877"}


Upload the file to S3 and save the path as a variable for later.

In [33]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://059124553121personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [34]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://059124553121personalizepocvod/


Now just make the call to kick off the batch export process.

In [36]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-HRNN2",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 25 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [37]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  05:32:23 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

Once the batch recommendations job has finished processing, we can grab the output uploaded to S3 and parse it.

In [38]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 5626,User: 2010,User: 78877
0,Interview with the Vampire: The Vampire Chroni...,Young Guns II (1990),"Truth About Cats & Dogs, The (1996)"
1,"Net, The (1995)",Young Guns (1988),Mighty Aphrodite (1995)
2,Waterworld (1995),Tombstone (1993),Phenomenon (1996)
3,Natural Born Killers (1994),Maverick (1994),Star Trek: First Contact (1996)
4,While You Were Sleeping (1995),Wyatt Earp (1994),"Nutty Professor, The (1996)"
5,Braveheart (1995),Unforgiven (1992),"Cable Guy, The (1996)"
6,GoldenEye (1995),Â¡Three Amigos! (1986),"Postman, The (Postino, Il) (1994)"
7,Quiz Show (1994),City Slickers II: The Legend of Curly's Gold (...,"River Wild, The (1994)"
8,"Firm, The (1993)",Last Man Standing (1996),Star Wars: Episode IV - A New Hope (1977)
9,Outbreak (1995),Legends of the Fall (1994),Nixon (1995)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `04_Clean_Up_Resources.ipynb`.

In [40]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)
